![Crisp](img/logo.png)
# <img src="img/anomaly_detection.svg" alt="icon" width="40"/> Anomaly Detection on Sales and Inventory Data

[![Open in Colab](https://img.shields.io/badge/Open%20in-Colab-orange?logo=google-colab&style=for-the-badge)](https://colab.research.google.com/github/gocrisp/blueprints/blob/main/notebooks/crisp_anomaly_detection.ipynb)
[![Open in Vertex AI](https://img.shields.io/badge/Open%20in-Vertex%20AI%20Workbench-brightgreen?logo=google-cloud&style=for-the-badge)](https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/gocrisp/blueprints/main/notebooks/crisp_anomaly_detection.ipynb)
[![Open in Databricks](https://img.shields.io/badge/Try-databricks-red?logo=databricks&style=for-the-badge)](https://www.databricks.com/try-databricks)
[![View on GitHub](https://img.shields.io/badge/View%20on-GitHub-lightgrey?logo=github&style=for-the-badge)](https://github.com/gocrisp/blueprints/blob/main/notebooks/crisp_anomaly_detection.ipynb)

> To deploy a notebook in Databricks:
> 1. Open your workspace and navigate to the folder where you want to import the notebook.
> 2. Click the triple-dot icon (next to the Share button).
> 3. Select Import and choose URL as the import method.
> 4. Paste the notebook's URL and click Import to complete the process.

Detecting anomalies in your data can help you identify unexpected patterns like stockouts, sales spikes, or pricing errors, so you can take action to improve inefficiencies, capitalize on gains, or address problems. This notebook demonstrates how to perform naive outlier detection on sales and inventory data using z-scores. We will identify data points that are considered anomalies based on the criterion that their z-score exceeds |3.5| within time windows of sufficient size.

## Set the required environment variables

In [1]:
import os

os.environ["ACCOUNT_ID"] = "999999"
# os.environ["CONNECTOR_ID"] = "" # optional

fact_sales_table = "exp_harmonized_retailer_fact_sales"
fact_inventory_table = "exp_harmonized_retailer_fact_inventory_store"

retailer = "target"

### Run Crisp common

This notebook uses the [crisp_common.ipynb](./crisp_common.ipynb) notebook to load the common functions and variables. The `crisp_common.ipynb` notebook contains the common functions and variables that are used across Crisp notebooks.

In [ ]:
import os

if not os.path.exists("crisp_common.ipynb"):
    print("Downloading crisp_common.ipynb")
    !wget https://raw.githubusercontent.com/gocrisp/blueprints/main/notebooks/crisp_common.ipynb -O crisp_common.ipynb
else:
    print("crisp_common.ipynb already exists")

%run crisp_common.ipynb

## Run extra imports

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
from IPython.display import display
import ipywidgets as widgets

### Load the data from BigQuery

We'll load data from the `fact_sales` and `fact_inventory` tables.

In [4]:
%%load sales_df
SELECT
    DATE(date_key) AS date,
    DATE_TRUNC(DATE(date_key), MONTH) AS time_window,
    store_id,
    product_id,
    sales_quantity
FROM
    `{project}`.`{dataset}`.`{fact_sales_table}`
WHERE
    sales_quantity IS NOT NULL
    AND retailer = '{retailer}'

In [5]:
%%load inventory_df
SELECT
    DATE(date_key) AS date,
    DATE_TRUNC(DATE(date_key), MONTH) AS time_window,
    store_id,
    product_id,
    on_hand_quantity AS inventory_quantity
FROM
    `{project}`.`{dataset}`.`{fact_inventory_table}`
WHERE
    on_hand_quantity IS NOT NULL
    AND retailer = '{retailer}'

### Perform anomaly detection using z-scores

For each time window (in this case each month) and product, we'll compute the z-score of `sales_quantity` and identify outliers where the absolute z-score exceeds 3.5.

In [ ]:
# Function to compute z-scores and identify outliers
def detect_outliers(group):
    if len(group) > 30:
        group["z_score"] = zscore(group["sales_quantity"])
        group["outlier"] = group["z_score"].abs() > 3.5
    else:
        group["z_score"] = np.nan
        group["outlier"] = False
    return group


sales_df["sales_quantity"] = sales_df["sales_quantity"].astype(float)

# Apply the function to each product within each time window
grouped_sales = (
    sales_df.groupby(["time_window", "product_id"], as_index=False)
    .apply(detect_outliers)
    .reset_index(drop=True)
)

We will then calculate the total number and percentage of anomalies in the dataset.


In [7]:
sales_anomalies = grouped_sales[grouped_sales["outlier"]]
sales_anomalies = (
    sales_anomalies.groupby(["product_id", "store_id", "time_window"])
    .last()
    .reset_index()
)

print(f"Number of anomalies detected in sales data: {len(sales_anomalies)}")
sales_anomalies.head()

print(f"Number of anomalies detected in sales data: {len(sales_anomalies)}")
print(f"Total number of sales data points: {len(sales_df)}")
print(
    f"Anomalies as a percentage of total sales data: {len(sales_anomalies) / len(sales_df) * 100:.2f}%"
)

Number of anomalies detected in sales data: 28891
Number of anomalies detected in sales data: 28891
Total number of sales data points: 5168400
Anomalies as a percentage of total sales data: 0.56%


### Visualize the anomalies in sales data

We will plot the sales quantities over time for a specific product and highlight the anomalies.

In [8]:
if not sales_anomalies.empty:
    # Prepare dropdown options
    options = []
    for _, row in sales_anomalies.iterrows():
        label = f"Product {row['product_id']} | Store {row['store_id']}"
        value = (row["product_id"], row["store_id"])
        options.append((label, value))

    # Create a dropdown widget
    product_dropdown = widgets.Dropdown(options=options, description="Select Product:")

    # Define a function to update the plot based on the selected product
    def update_sales_plot(product_and_store):
        product_id, store_id = product_and_store
        product_data = grouped_sales[
            (grouped_sales["product_id"] == product_id)
            & (grouped_sales["store_id"] == store_id)
        ].copy()
        if product_data.empty:
            print(f"No data available for Product {product_id} and Store {store_id}.")
            return
        # Ensure 'outlier' is boolean
        product_data["outlier"] = product_data["outlier"].astype(bool)

        # Plot sales quantity over time
        plt.figure(figsize=(12, 6))
        sns.lineplot(
            data=product_data, x="date", y="sales_quantity", label="Sales Quantity"
        )

        # Plot anomalies
        anomalies = product_data[product_data["outlier"]]
        sns.scatterplot(
            data=anomalies,
            x="date",
            y="sales_quantity",
            color="red",
            label="Anomalies",
            s=100,
        )
        plt.title(
            f"Sales Quantity Over Time for Product {product_id} | Store {store_id}"
        )
        plt.xlabel("Date")
        plt.ylabel("Sales Quantity")
        plt.legend()
        display(plt.show())

    # Create an interactive widget to update the plot
    interactive_plot = widgets.interactive(
        update_sales_plot, product_and_store=product_dropdown
    )
    display(interactive_plot)
else:
    print("No anomalies detected to visualize.")

interactive(children=(Dropdown(description='Select Product:', options=(('Product 101 | Store 1256909618105974'…

### Perform anomaly detection on inventory data (if needed)

You can apply the same process to the inventory data.

In [ ]:
# Function to compute z-scores and identify outliers
def detect_outliers_inventory(group):
    if len(group) > 30:
        group["z_score"] = zscore(group["inventory_quantity"])
        group["outlier"] = group["z_score"].abs() > 3.5
    else:
        group["z_score"] = np.nan
        group["outlier"] = False
    return group


inventory_df["inventory_quantity"] = inventory_df["inventory_quantity"].astype(float)

# Apply the function to each product within each time window
grouped_inventory = (
    inventory_df.groupby(["time_window", "product_id"], as_index=False)
    .apply(detect_outliers_inventory)
    .reset_index(drop=True)
)

### Examine the anomalies in inventory data

In [10]:
inventory_anomalies = grouped_inventory[grouped_inventory["outlier"]]
inventory_anomalies = (
    inventory_anomalies.groupby(["product_id", "store_id", "time_window"])
    .last()
    .reset_index()
)

print(f"Number of anomalies detected in inventory data: {len(inventory_anomalies)}")
inventory_anomalies.head()

print(f"Number of anomalies detected in inventory data: {len(inventory_anomalies)}")
print(f"Total number of inventory data points: {len(inventory_df)}")
print(
    f"Anomalies as a percentage of total inventory data: {len(inventory_anomalies) / len(inventory_df) * 100:.2f}%"
)

Number of anomalies detected in inventory data: 8607
Number of anomalies detected in inventory data: 8607
Total number of inventory data points: 5168400
Anomalies as a percentage of total inventory data: 0.17%


### Visualize the anomalies in inventory data

In [11]:
if not inventory_anomalies.empty:
    # Prepare dropdown options
    options = []
    for _, row in inventory_anomalies.iterrows():
        label = f"Product {row['product_id']} | Store {row['store_id']}"
        value = (row["product_id"], row["store_id"])
        options.append((label, value))

    # Create a dropdown widget
    product_dropdown = widgets.Dropdown(options=options, description="Select Product:")

    # Define a function to update the plot based on the selected product
    def update_inventory_plot(product_and_store):
        product_id, store_id = product_and_store
        product_data = grouped_inventory[
            (grouped_inventory["product_id"] == product_id)
            & (grouped_inventory["store_id"] == store_id)
        ].copy()

        if product_data.empty:
            print(f"No data available for Product {product_id} and Store {store_id}.")
            return

        # Ensure 'outlier' is boolean
        product_data["outlier"] = product_data["outlier"].astype(bool)

        # Plot inventory quantity over time
        plt.figure(figsize=(12, 6))
        sns.lineplot(
            data=product_data,
            x="date",
            y="inventory_quantity",
            label="Inventory Quantity",
        )

        # Plot anomalies
        anomalies = product_data[product_data["outlier"]]
        sns.scatterplot(
            data=anomalies,
            x="date",
            y="inventory_quantity",
            color="red",
            label="Anomalies",
            s=100,
        )
        plt.title(
            f"Inventory Quantity Over Time for Product {product_id} | Store {store_id}"
        )
        plt.xlabel("Date")
        plt.ylabel("Inventory Quantity")
        plt.legend()
        display(plt.show())

    # Create an interactive widget to update the plot
    interactive_plot = widgets.interactive(
        update_inventory_plot, product_and_store=product_dropdown
    )
    display(interactive_plot)
else:
    print("No anomalies detected to visualize.")

interactive(children=(Dropdown(description='Select Product:', options=(('Product 101 | Store 7243408426042915'…

### Save or report the anomalies

#### Save anomalies to CSV files

In [12]:
# Save sales anomalies to a CSV file
sales_anomalies.to_csv("sales_anomalies.csv", index=False)
print("Sales anomalies saved to 'sales_anomalies.csv'.")

# Save inventory anomalies to a CSV file
inventory_anomalies.to_csv("inventory_anomalies.csv", index=False)
print("Inventory anomalies saved to 'inventory_anomalies.csv'.")

Sales anomalies saved to 'sales_anomalies.csv'.
Inventory anomalies saved to 'inventory_anomalies.csv'.


#### Save anomalies to your data warehouse

In [13]:
sales_anomalies

,product_id,store_id,time_window,date,sales_quantity,z_score,outlier
0,101,1256909618105974,2024-01-01,2024-01-02,6.0,3.521871,True
1,101,1256909618105974,2024-04-01,2024-04-21,6.0,3.604485,True
2,101,1256909618105974,2024-05-01,2024-05-04,6.0,3.637330,True
3,101,1256909618105974,2024-08-01,2024-08-20,6.0,3.600564,True
4,101,2893058682334322,2024-04-01,2024-04-24,6.0,3.604485,True
...,...,...,...,...,...,...,...
28886,180,974851238042301426,2024-05-01,2024-05-10,6.0,3.564098,True
28887,180,974851238042301426,2024-07-01,2024-07-14,6.0,3.698851,True
28888,180,974851238042301426,2024-11-01,2024-11-06,7.0,4.395046,True
28889,180,979316279447426146,2024-05-01,2024-05-06,8.0,5.005446,True


In [14]:
%%save {project}.{dataset}.sales_anomalies
sales_anomalies

In [15]:
inventory_anomalies

,product_id,store_id,time_window,date,inventory_quantity,z_score,outlier
0,101,7243408426042915,2024-10-01,2024-10-28,35.0,3.639414,True
1,101,36297635338677003,2024-06-01,2024-06-02,37.0,4.004743,True
2,101,51254687980601498,2024-04-01,2024-04-12,42.0,4.081629,True
3,101,53567763105050586,2024-10-01,2024-10-31,38.0,4.012603,True
4,101,53567763105050586,2024-11-01,2024-11-05,46.0,4.613783,True
...,...,...,...,...,...,...,...
8602,180,938296014196475287,2024-06-01,2024-06-17,43.0,4.211624,True
8603,180,938296014196475287,2024-07-01,2024-07-04,43.0,3.953940,True
8604,180,942485310238374410,2024-01-01,2024-01-11,32.0,3.551931,True
8605,180,942485310238374410,2024-08-01,2024-08-30,37.0,3.785531,True


In [16]:
%%save {project}.{dataset}.inventory_anomalies
inventory_anomalies